In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_context('notebook', font_scale= 1.2)

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, classification_report

import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']

In [5]:
# data = pd.read_csv('/Users/mucheng/Datasets/DataCoSupplyChainDatasetplus.csv', encoding='ISO-8859-1')
# data = pd.read_csv('DataCoSupplyChainDatasetplus(1).csv', encoding='ISO-8859-1')
data = pd.read_csv('/Users/mucheng/Desktop/Desktop\ Storage/DataCoSupplyChainDataset.csv', encoding='ISO-8859-1')
data_past = pd.read_csv('/Users/mucheng/Desktop/Desktop\ Storage/DataCoSupplyChainDataset.csv', encoding='ISO-8859-1')
data
data_copy = data.copy()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mucheng/Desktop/Desktop\\ Storage/DataCoSupplyChainDataset.csv'

In [ ]:
#reading the raw data
data_total = pd.read_excel('data/DataCoSupplyChainDataset_raw1.xlsx')

In [ ]:
data_total

In [ ]:
#explore the distribution of each features' data
for i in data_total.columns:
    print(data_total[i].value_counts())

In [ ]:
data_total_copy = data_total.copy()
data_total_copy.drop(['Product Name', 'Order Profit Per Order','Order Country',  'Order Status','Customer Email',
                      'order date (DateOrders)', 'Customer Password', 'Product Category Id', 'Product Image',
                      'Product Status', 'Product Price', 'shipping date (DateOrders)', 'Customer State', 'Order Customer Id'], axis=1, inplace=True)

In [ ]:
# Check which features has too many null values
data_total_copy.info()
# drop the following features：
# Product Description
# Order Zipcode

In [ ]:
# after the aobe step, we handpicked features, and kept 20 features for analysis

In [ ]:
from sklearn.preprocessing import LabelEncoder


for i in  ['Days for shipping (real)', 'Delivery Status', 'Category Name', 'Customer Country','Order Country','Customer City','Customer Fname', 'Customer Lname', 'Customer Segment', 'Order City']:
    le = LabelEncoder()
    data_past[i] = le.fit_transform(data_past[i])

# 'Category Name', 'Customer Country','Order Country'
# category


class_sm = {
    'Standard Class' : 3,
    'Second Class' : 2,
    'First Class' : 1,
    'Same Day' : 0
}

market_dict ={
    "Pacific Asia" : 1,
    "USCA" : 2,
    "Africa" : 3,
    "Europe" : 4,
    "LATAM" : 5  
}

data_df_past = data_past.dropna().reset_index()
data_df_past['Shipping Mode'] = data_df_past['Shipping Mode'].map(class_sm)
data_df_past['Shipping Mode'] = data_df['Shipping Mode'].apply(np.int64)
data_df_past['Market'] = data_df_past['Market'].map(market_dict)
data_df_past['Market'] = data_df_past['Market'].apply(np.int64)
del data_df_past['index']
data_df_past.head()

In [ ]:
# categroy name & customer country & order country
count = 0
sum_order_country = 0
for i in data_df['Order Country'].value_counts():
    if i > 1000:
        count += 1
        sum_order_country += i
sum_order_country

In [ ]:
147778 / len(data)

In [ ]:
# spearman correlation analysis
fig, ax = plt.subplots(figsize=(15,15))
sns.heatmap(data_df_past[[i for i in data_df_past.columns if i not in ['Benefit per order', 'Late_delivery_risk']]].corr(method="spearman"),square=True,annot=True,ax=ax,annot_kws={'fontsize':15})
plt.xticks(fontsize=15,rotation=270)
plt.yticks(fontsize=15,rotation=360)
plt.show()

In [ ]:
df = data_df_past[[i for i in data_df_past.columns if i not in ['Benefit per order', 'Late_delivery_risk']]].corr(method="spearman")

In [ ]:
# Mapping features then filter features in pairs
# find pairs which points in heat map is between 0.1 to 1
# (19*19/2 pairs as total)
high = 1
low = 0.1
result = []
for i in df.columns:
    df_0 = pd.DataFrame(df[i].sort_values(ascending=False))
    indexs = list(df_0[df_0[i] > low][df_0[i] < high].index)
#     print(indexs)
    if len(indexs)>0:
        for j in range(len(indexs)):
            result.append([i, indexs[j]])
print(result)

## Predict Benefit and Risk Per Order

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score, accuracy_score

In [ ]:
# predicting benefit per order based on trainning the existing data 
# (when we do the prediction, we have taken out features 'Benefit per order', 'Late_delivery_risk' from data
# and test size = 0.25)

X_train, X_test, y_train, y_test = train_test_split(data_df_past[[i for i in data_df_past.columns if i not in ['Benefit per order', 'Late_delivery_risk']]], data_df['Benefit per order'], test_size=0.25)

In [ ]:
XG_model = XGBRegressor()
XG_model.fit(X_train, y_train)
y_pred = XG_model.predict(X_test)
# print('lgb mse', mean_squared_error(y_test, y_pred))

In [ ]:
# Connecting the trainning data's feature 'Shipping Mode' with 
# predicting feature 'benefit per order'

y_pred = pd.DataFrame(y_pred, columns={'benefit'})
X_test = X_test.reset_index()
del X_test['index']
benefit_new_df = pd.concat([X_test, y_pred], axis=1)
benefit_new_df = benefit_new_df[['Shipping Mode', 'benefit']]
benefit_new_df

In [ ]:
# 'Benefit per order' predicted in different shipping mode
# and their related datas
benefit_new_df.groupby(["Shipping Mode"]).describe()

In [ ]:
# risk -- risk of being late
# predicting risk based on trainning the existing data 
# (when we do the prediction, we have taken out features 'Benefit per order', 'Late_delivery_risk' from data
# and test size = 0.25)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_df_past[[i for i in data_df_past.columns if i not in ['Benefit per order', 'Late_delivery_risk']]], data_df['Late_delivery_risk'], test_size=0.25)

LR_risk_model = LogisticRegression()
LR_risk_model.fit(X_train, y_train)
y_pred_01 = LR_risk_model.predict(X_test)
print('lgb acc', accuracy_score(y_test, y_pred_01))
y_pred = LR_risk_model.predict_proba(X_test)[:, 1]
print('lgb auc', roc_auc_score(y_test, y_pred))


In [ ]:
y_pred = pd.DataFrame(y_pred_01, columns={'risk_rate'})
X_test = X_test.reset_index()
del X_test['index']
risk_new_df = pd.concat([X_test, y_pred], axis=1)
risk_new_df = risk_new_df[['Shipping Mode', 'risk_rate']]
risk_new_df.head()

In [ ]:
risk_new_df.groupby(['Shipping Mode']).describe()

In [ ]:
data = pd.read_csv('data/DataCoSupplyChainDatasetplus.csv', encoding='ISO-8859-1')
data.head()

In [ ]:
data['Days of Shipping(schedule-real)'].value_counts()

In [ ]:
# 1--advance, 0--on time, -1--late
def process_real_sch(data):
    if data > 0:
        return 1
    elif data == 0:
        return 0
    else:
        return -1
data['advance_ontime_late'] = data['Days of Shipping(schedule-real)'].apply(process_real_sch)

In [ ]:
data['advance_ontime_late'].value_counts()

In [ ]:
data_real_sch = data[['Days of Shipping(schedule-real)', 'advance_ontime_late', 'Benefit per order']]

In [ ]:
# Correlation matrix
# Benefit per order is not highly related to whether the shipping is advance, on time or late.
data_real_sch.corr()

In [ ]:
data_real_sch.describe()

In [ ]:
# Density Plot 1 showing the distribution of feature 'Days of Shipping(schedule-real)' when shipping is in advance,
# on-time and late. 
# Density Plot 2 showing the distribution of feature 'Benefit per order' when shipping is in advance, on-time and late.   

fig, ax = plt.subplots(2, 1, figsize = (20, 15))
plt.suptitle('advance_ontime_late', fontsize = 25, color = 'teal')
sns.kdeplot(x = data_real_sch['Days of Shipping(schedule-real)'], hue= data_real_sch['advance_ontime_late'], shade= True, ax= ax[0], palette = 'viridis')
plt.suptitle('advance_ontime_late', fontsize = 25, color = 'teal')
sns.kdeplot(x = data_real_sch['Benefit per order'], hue= data_real_sch['advance_ontime_late'], shade= True, ax= ax[1], palette = 'viridis')
plt.show()


In [ ]:
# When shipping arrives in advance, what is the trend of 'Days of Shipping(schedule-real)' and 'Benefit per order'.
data_high = data_real_sch[data_real_sch['advance_ontime_late'] == 1]

# Density Plot 3 showing the distribution of feature 'Days of Shipping(schedule-real)' when shipping is in advance
# Density Plot 4 showing the distribution of feature 'Benefit per order' when shipping is in advance 
# (Most of the shipping status is in advance for 1 or 2 days, and 81.33% of those transactions' benefit is non-negative)
fig, ax = plt.subplots(2, 1, figsize = (20, 15))

plt.suptitle('advance_ontime_late', fontsize = 25, color = 'teal')
sns.kdeplot(x = data_high['Days of Shipping(schedule-real)'], hue= data_high['advance_ontime_late'], shade= True, ax= ax[0], palette = 'viridis')
plt.suptitle('advance_ontime_late', fontsize = 25, color = 'teal')
sns.kdeplot(x = data_high['Benefit per order'], hue= data_high['advance_ontime_late'], shade= True, ax= ax[1], palette = 'viridis')
plt.show()


In [ ]:
# When the shipping is in advance, the percentage of benefit being non-negative.
print('{:.2%}'.format(len(data_high[data_high['Benefit per order'] >= 0]) / len(data_high)))

In [ ]:
# When shipping arrives on time, what is the trend of 'Days of Shipping(schedule-real)' and 'Benefit per order'.
data_eql = data_real_sch[data_real_sch['advance_ontime_late'] == 0]

# Density Plot 5 showing the distribution of feature 'Days of Shipping(schedule-real)' when shipping arrives on time
# (81.43% of the benefit is still greater or equal to 0)
fig, ax = plt.subplots(1,1,figsize = (20, 15))
plt.suptitle('advance_ontime_late', fontsize = 25, color = 'teal')
sns.kdeplot(x = data_eql['Benefit per order'], hue= data_eql['advance_ontime_late'], shade= True, ax= ax, palette = 'viridis')

plt.show()


In [ ]:
# When shipping arrives on time, what is the percentage of benefit per order is non-negative.
print('{:.2%}'.format(len(data_eql[data_eql['Benefit per order'] >= 0]) / len(data_eql)))

In [ ]:
# When the shipping is late, the percentage of benefit being non-negative.
data_low = data_real_sch[data_real_sch['advance_ontime_late'] == -1]

# Density Plot 6 showing the distribution of feature 'Days of Shipping(schedule-real)' when shipping is late
# Density Plot 7 showing the distribution of feature 'Benefit per order' when shipping is late 
# (Most of the shipping status is in advance for 1 to 4 days, and 80.57% of those transactions' benefit is non-negative)

fig, ax = plt.subplots(2, 1, figsize = (20, 15))

plt.suptitle('advance_ontime_late', fontsize = 25, color = 'teal')
sns.kdeplot(x = data_low['Days of Shipping(schedule-real)'], hue= data_low['advance_ontime_late'], shade= True, ax= ax[0], palette = 'viridis')
plt.suptitle('advance_ontime_late', fontsize = 25, color = 'teal')
sns.kdeplot(x = data_low['Benefit per order'], hue= data_low['advance_ontime_late'], shade= True, ax= ax[1], palette = 'viridis')
plt.show()

In [ ]:
# When shipping is late, what is the percentage of benefit per order being non-negative?
print('{:.2%}'.format(len(data_low[data_low['Benefit per order'] >= 0]) / len(data_low)))

In [ ]:
# Exploring the relationships between shipping result(advance/on time/late) and benefit
# (generated new feature "benefit_get", if benefit >= 0, benefit_get = 1; if benefit >= 0, benefit_get = 0)
data_real_sch.head()

In [ ]:
data_real_sch['benefit_get'] = data_real_sch['Benefit per order'].apply(lambda x:1 if x >= 0 else 0)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (20, 15))

# Density Plot 8 showing the distribution of shipping status is in advance, on-time and late
# (when feature 'Days of Shipping(schedule-real)' > 0, = 0, < 0, respectively), and 
# the distribution of benefit_get is equal to 1 and 0 (when feature 'Benefit per order' < 0 and >= 0)

plt.suptitle('advance_ontime_late', fontsize = 25, color = 'teal')
sns.kdeplot(x = data_real_sch['advance_ontime_late'], hue= data_real_sch['benefit_get'], shade= True, ax= ax, palette = 'viridis')

In [ ]:
data_real_sch.head()

In [ ]:
# Relationships b/w benefit_get and shipping result 

# benefit_get = 1 ---> shipping %
# when benefit per order is non-negative, what is the percentage of shipping is 
# in advance, on-time and late respectively
print(data_real_sch[data_real_sch['benefit_get'] == 1]['advance_ontime_late'].value_counts()[1]/len(data_real_sch[data_real_sch['benefit_get'] == 1]))
print(data_real_sch[data_real_sch['benefit_get'] == 1]['advance_ontime_late'].value_counts()[0]/len(data_real_sch[data_real_sch['benefit_get'] == 1]))
print(data_real_sch[data_real_sch['benefit_get'] == 1]['advance_ontime_late'].value_counts()[-1]/len(data_real_sch[data_real_sch['benefit_get'] == 1]))

In [ ]:
# benefit_get = 0 ---> shipping %
# when benefit per order is negative, what is the percentage of shipping is in advance, 
# on-time and late respectively
print(data_real_sch[data_real_sch['benefit_get'] == 0]['advance_ontime_late'].value_counts()[1]/len(data_real_sch[data_real_sch['benefit_get'] == 0]))
print(data_real_sch[data_real_sch['benefit_get'] == 0]['advance_ontime_late'].value_counts()[0]/len(data_real_sch[data_real_sch['benefit_get'] == 0]))
print(data_real_sch[data_real_sch['benefit_get'] == 0]['advance_ontime_late'].value_counts()[-1]/len(data_real_sch[data_real_sch['benefit_get'] == 0]))

In [ ]:
# Figuring out the company's requent customers
from collections import Counter
data_customID = data['Customer Id']
id_counts = Counter(data_customID)

In [ ]:
# make feature 'Customer Id' and their frequency become a dict.
# {(customer id 1, frequency),customer id 2, frequency),..}
id_counts = dict(id_counts)

# make dict id_counts become tuple
# and sort the value(frequency) in descending order (from high to low)
id_counts = sorted(id_counts.items(), key=lambda x: -x[1])

In [ ]:
# Keep feature 'Customer Id' and their frequency in 
# two arrays respectively to get each customer ID's frequency in percentage
# id_counts_x：'Customer Id' ；id_counts_y：frequency 
id_counts_x = []
id_counts_y = []
for i in id_counts:
    id_counts_x.append(i[0])
    id_counts_y.append(i[1])
# print(len(id_counts_x))

In [ ]:
# chart showing for each customer ID, 
# what is their frequency and their percentage for all transactions 
data_custom_ID = pd.DataFrame({"ID" :id_counts_x, "times":id_counts_y}) 
data_custom_ID['rate'] = data_custom_ID['times'].apply(lambda x:str(100. * x/len(data))[:6] + "%")
data_custom_ID.head(10)

In [ ]:
# Create a pie chart for top 100 customers 
# tags for pie chart: customer ID -- frequency of this ID in the data
data_custom = pd.DataFrame({'custom':id_counts_x, 'times':id_counts_y})
data_custom['times'] = data_custom['times'].apply(lambda x:np.nan if x in [1] else x)
data_custom = data_custom.dropna().reset_index().drop('index',axis=1)

In [ ]:
# Generating pie chart
from pyecharts.charts import Pie, Bar
from pyecharts.render import make_snapshot
from pyecharts import options as opts
warnings.filterwarnings('ignore')
p_dict = {'width':'1920px', 'height': '1080px'}
c = (
    Pie(init_opts=opts.InitOpts(width=p_dict['width'], height=p_dict['height']))
        .add("", [list(z) for z in zip(data_custom['custom'][:100], data_custom['times'][:100])], center=["40%", "50%"]) # avoid sheltering
        .set_global_opts(title_opts=opts.TitleOpts(title="custom_ID"))  # tags
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))  # data tags setting
)

c.render('ID.html')


## When shipping status is late, what is the distribution of 
## category name, customer country and order country (1.a/b/c)
## and the distribution of 1 - 4 days respectively (2.a/b/c).

#### 1.Exploring when shipping status is late, the distribution of 
#### category name, customer country and order country. (1.a/b/c)

In [ ]:
data_total = pd.read_csv('data/DataCoSupplyChainDatasetplus.csv',encoding='ISO-8859-1')

In [ ]:
data_total.head()

In [ ]:
out_date = data_total[data_total['Days of Shipping(schedule-real)'] < 0].reset_index().drop('index',axis=1)
out_date.head()

In [ ]:
# 1.a. when shipping status is late, the distribution of category name, customer country and order country.
total_cn = pd.DataFrame(data_total['Category Name'].value_counts()).reset_index()
out_cn = pd.DataFrame(out_date['Category Name'].value_counts()).reset_index()
total = pd.merge(total_cn, out_cn, on='index', how = 'left')

total.columns = ['category name', 'total', 'late order quantity']
total.head()

In [ ]:
total['rate'] = total['late order quantity'] / total['total']
total = total.sort_values('rate', ascending=False).reset_index().drop('index', axis=1)
total.head(5)

In [ ]:
# bar chart total category name late
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['category name'][i],total['rate'][i])
    
plt.title("late & category name")
plt.xticks(rotation=90)  
plt.xlabel("category")
plt.ylabel("rate")
 
plt.show()

In [ ]:
# 1.b. when shipping status is late, the distribution of customer country
# (There are only 2 customer country as a total)
out_date['Customer Country'].value_counts()

In [ ]:
total_cc = pd.DataFrame(data_total['Customer Country'].value_counts()).reset_index()
out_cc = pd.DataFrame(out_date['Customer Country'].value_counts()).reset_index()
total = pd.merge(total_cc, out_cc, on='index', how = 'left')
total.columns = ['Customer Country', 'total', 'late order quantity']
total.head()

In [ ]:
total['rate'] = total['late order quantity'] / total['total']
total = total.sort_values('rate', ascending=False).reset_index().drop('index', axis=1)
total.head()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['Customer Country'][i],total['rate'][i])
    
plt.title("late & custom country")
plt.xticks(rotation=90) 
plt.xlabel("Customer Country")
plt.ylabel("rate")
 
plt.show()

In [ ]:
# 1.c. when shipping status is late, the distribution of order country.
# (Before doing the analysis, since there are too many type of countries in this feature,
# We need to scan out those countries that have more than 1000 orders with the company, 
# and then do the analysis.)

total_oc = pd.DataFrame(data_total['Order Country'].value_counts()).reset_index()
count = 0
sum_order_country = 0
result = []
for i in range(len(total_oc)):
    if total_oc['Order Country'][i] > 1000:
        count += 1
        sum_order_country += total_oc['Order Country'][i]
        result.append(total_oc['index'][i])
sum_order_country, count, sum_order_country/len(data)

In [ ]:
# total amount of order country counted into analysis.

total_cn = pd.DataFrame(data_total['Order Country'].value_counts()).reset_index()
out_cn = pd.DataFrame(out_date['Order Country'].value_counts()).reset_index()
out_cn

In [ ]:
total_cn = pd.DataFrame(data_total['Order Country'].value_counts()).reset_index()
out_cn = pd.DataFrame(out_date['Order Country'].value_counts()).reset_index()

for i in range(len(out_cn)):
    if out_cn['index'][i] not in result:
        out_cn['Order Country'][i] = np.nan
out_cn = out_cn.dropna().reset_index().drop('level_0',axis=1)
total = pd.merge(total_cn, out_cn[['index', 'Order Country']], on='index', how = 'right')
total.columns = ['Order Country', 'total', 'late order quantity']
total.head()

In [ ]:
total['rate'] = total['late order quantity'] / total['total']
total = total.sort_values('rate', ascending=False).reset_index().drop('index', axis=1)
total.head()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['Order Country'][i],total['rate'][i])
    
plt.title("late & Order Country")
plt.xticks(rotation=90)  
plt.xlabel("Order Country")
plt.ylabel("rate")

plt.show()

#####  2. When shipping status is late, the distribution of  1-4 late days of 
#### category name, customer country and order country (2.a/b/c). 

In [ ]:
out_date['Days of Shipping(schedule-real)'].value_counts()

In [ ]:
late1 = out_date[out_date['Days of Shipping(schedule-real)'] == -1]
late2 = out_date[out_date['Days of Shipping(schedule-real)'] == -2]
late3 = out_date[out_date['Days of Shipping(schedule-real)'] == -3]
late4 = out_date[out_date['Days of Shipping(schedule-real)'] == -4]

##### 2.a.  When shipping status is late, the distribution of 1 - 4 late days of category name
##### (The bar charts below is from 1st late day to 4th late day respectively)

In [ ]:
total_cn = pd.DataFrame(data_total['Category Name'].value_counts()).reset_index()
out_cn = pd.DataFrame(late1['Category Name'].value_counts()).reset_index()
total = pd.merge(total_cn, out_cn, on='index', how = 'left')
total.columns = ['category name', 'total', 'out']
total['rate'] = total['out'] / total['total']
total = total.sort_values('rate', ascending=False).reset_index().drop('index', axis=1)
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['category name'][i],total['rate'][i])
    
plt.title("late & category name")
plt.xticks(rotation=90)  
plt.xlabel("category")
plt.ylabel("rate")

plt.show()

In [ ]:
total_cn = pd.DataFrame(data_total['Category Name'].value_counts()).reset_index()
out_cn = pd.DataFrame(late2['Category Name'].value_counts()).reset_index()
total = pd.merge(total_cn, out_cn, on='index', how = 'left')
total.columns = ['category name', 'total', 'out']
total['rate'] = total['out'] / total['total']
total = total.sort_values('rate', ascending=False).reset_index().drop('index', axis=1)
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['category name'][i],total['rate'][i])
    
plt.title("late & category name")
plt.xticks(rotation=90)  
plt.xlabel("category")
plt.ylabel("rate")

plt.show()

In [ ]:
total_cn = pd.DataFrame(data_total['Category Name'].value_counts()).reset_index()
out_cn = pd.DataFrame(late3['Category Name'].value_counts()).reset_index()
total = pd.merge(total_cn, out_cn, on='index', how = 'left')
total.columns = ['category name', 'total', 'out']
total['rate'] = total['out'] / total['total']
total = total.sort_values('rate', ascending=False).reset_index().drop('index', axis=1)
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['category name'][i],total['rate'][i])
    
plt.title("late & category name")
plt.xticks(rotation=90) 
plt.xlabel("category")
plt.ylabel("rate")

plt.show()

In [ ]:
total_cn = pd.DataFrame(data_total['Category Name'].value_counts()).reset_index()
out_cn = pd.DataFrame(late4['Category Name'].value_counts()).reset_index()
total = pd.merge(total_cn, out_cn, on='index', how = 'left')
total.columns = ['category name', 'total', 'out']
total['rate'] = total['out'] / total['total']
total = total.sort_values('rate', ascending=False).reset_index().drop('index', axis=1)
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['category name'][i],total['rate'][i])
    
plt.title("late & category name")
plt.xticks(rotation=90) 
plt.xlabel("category")
plt.ylabel("rate")

plt.show()

##### 2.b.  When shipping status is late, the distribution of 1 - 4 late days of customer country of late orders
##### (The bar charts below is from 1st late day to 4th late day respectively, since there is only
##### 2 countries, unlike all the other analysis, we regard the total as late orders (instead of all orders including late, on-time and 
##### in advance). So the bar charts below are showing how distribution of order changes in late shipping status
##### from day 1 to day 4.


In [ ]:
total_cn = pd.DataFrame(data_total['Customer Country'].value_counts()).reset_index()
out_cn = pd.DataFrame(late1['Customer Country'].value_counts()).reset_index()
total = pd.merge(total_cn, out_cn, on='index', how = 'left')
total.columns = ['Customer Country', 'total', 'out']
total['rate'] = total['out'] / sum(total['out'])
total = total.sort_values('rate', ascending=False).reset_index().drop('index', axis=1)
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['Customer Country'][i],total['rate'][i])
    
plt.title("late & Customer Country")
plt.xticks(rotation=90) 
plt.xlabel("Customer Country")
plt.ylabel("rate")

plt.show()

In [ ]:
total_cn = pd.DataFrame(data_total['Customer Country'].value_counts()).reset_index()
out_cn = pd.DataFrame(late2['Customer Country'].value_counts()).reset_index()
total = pd.merge(total_cn, out_cn, on='index', how = 'left')
total.columns = ['Customer Country', 'total', 'out']
total['rate'] = total['out'] / sum(total['out'])
total = total.sort_values('rate', ascending=False).reset_index().drop('index', axis=1)
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['Customer Country'][i],total['rate'][i])
    
plt.title("late & Customer Country")
plt.xticks(rotation=90) 
plt.xlabel("Customer Country")
plt.ylabel("rate")

plt.show()

In [ ]:
total_cn = pd.DataFrame(data_total['Customer Country'].value_counts()).reset_index()
out_cn = pd.DataFrame(late3['Customer Country'].value_counts()).reset_index()
total = pd.merge(total_cn, out_cn, on='index', how = 'left')
total.columns = ['Customer Country', 'total', 'out']
total['rate'] = total['out'] / sum(total['out'])
total = total.sort_values('rate', ascending=False).reset_index().drop('index', axis=1)

import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['Customer Country'][i],total['rate'][i])
    
plt.title("late & Customer Country")
plt.xticks(rotation=90)  
plt.xlabel("Customer Country")
plt.ylabel("rate")

plt.show()

In [ ]:
total_cn = pd.DataFrame(data_total['Customer Country'].value_counts()).reset_index()
out_cn = pd.DataFrame(late4['Customer Country'].value_counts()).reset_index()
total = pd.merge(total_cn, out_cn, on='index', how = 'left')
total.columns = ['Customer Country', 'total', 'out']

total['rate'] = total['out'] / sum(total['out'])
total = total.sort_values('rate', ascending=False).reset_index().drop('index', axis=1)
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['Customer Country'][i],total['rate'][i])
    
plt.title("late & Customer Country")
plt.xticks(rotation=90)  
plt.xlabel("Customer Country")
plt.ylabel("rate")

plt.show()

##### 2.c.  When shipping status is late, the distribution of 1 - 4 late days of order country
##### (The bar charts below is from 1st late day to 4th late day respectively)

In [ ]:

total_cn = pd.DataFrame(data_total['Order Country'].value_counts()).reset_index()
out_cn = pd.DataFrame(late1['Order Country'].value_counts()).reset_index()
# out_cn['Order Country1'] = out_cn['index'].apply(lambda x:x if x in result else np.nan)
for i in range(len(out_cn)):
    if out_cn['index'][i] not in result:
        out_cn['Order Country'][i] = np.nan
out_cn = out_cn.dropna().reset_index().drop('level_0',axis=1)
total = pd.merge(total_cn, out_cn[['index', 'Order Country']], on='index', how = 'right')

total.columns = ['Order Country', 'total', 'out']
total['rate'] = total['out'] / sum(total['out'])
total = total.sort_values('rate', ascending=False).reset_index().drop('index', axis=1)
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['Order Country'][i],total['rate'][i])
    
plt.title("late & Order Country")
plt.xticks(rotation=90) 
plt.xlabel("Order Country")
plt.ylabel("rate")

plt.show()

In [ ]:
total_cn = pd.DataFrame(data_total['Order Country'].value_counts()).reset_index()
out_cn = pd.DataFrame(late2['Order Country'].value_counts()).reset_index()
# out_cn['Order Country1'] = out_cn['index'].apply(lambda x:x if x in result else np.nan)
for i in range(len(out_cn)):
    if out_cn['index'][i] not in result:
        out_cn['Order Country'][i] = np.nan
out_cn = out_cn.dropna().reset_index().drop('level_0',axis=1)
total = pd.merge(total_cn, out_cn[['index', 'Order Country']], on='index', how = 'right')

total.columns = ['Order Country', 'total', 'out']
total['rate'] = total['out'] / sum(total['out'])
total = total.sort_values('rate', ascending=False).reset_index().drop('index', axis=1)
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['Order Country'][i],total['rate'][i])
    
plt.title("late & Order Country")
plt.xticks(rotation=90) 
plt.xlabel("Order Country")
plt.ylabel("rate")

plt.show()

In [ ]:
total_cn = pd.DataFrame(data_total['Order Country'].value_counts()).reset_index()
out_cn = pd.DataFrame(late3['Order Country'].value_counts()).reset_index()
# out_cn['Order Country1'] = out_cn['index'].apply(lambda x:x if x in result else np.nan)
for i in range(len(out_cn)):
    if out_cn['index'][i] not in result:
        out_cn['Order Country'][i] = np.nan
out_cn = out_cn.dropna().reset_index().drop('level_0',axis=1)
total = pd.merge(total_cn, out_cn[['index', 'Order Country']], on='index', how = 'right')

total.columns = ['Order Country', 'total', 'out']
total['rate'] = total['out'] / sum(total['out'])
total = total.sort_values('rate', ascending=False).reset_index().drop('index', axis=1)
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['Order Country'][i],total['rate'][i])
    
plt.title("late & Order Country")
plt.xticks(rotation=90) 
plt.xlabel("Order Country")
plt.ylabel("rate")

plt.show()

In [ ]:
total_cn = pd.DataFrame(data_total['Order Country'].value_counts()).reset_index()
out_cn = pd.DataFrame(late4['Order Country'].value_counts()).reset_index()
# out_cn['Order Country1'] = out_cn['index'].apply(lambda x:x if x in result else np.nan)
for i in range(len(out_cn)):
    if out_cn['index'][i] not in result:
        out_cn['Order Country'][i] = np.nan
out_cn = out_cn.dropna().reset_index().drop('level_0',axis=1)
total = pd.merge(total_cn, out_cn[['index', 'Order Country']], on='index', how = 'right')

total.columns = ['Order Country', 'total', 'out']
total['rate'] = total['out'] / sum(total['out'])
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
for i in range(len(total)):
    plt.bar(total['Order Country'][i],total['rate'][i])
    
plt.title("late & Order Country")
plt.xticks(rotation=90) 
plt.xlabel("Order Country")
plt.ylabel("rate")

plt.show()